In [3]:
import os
import pandas as pd

df = pd.read_csv("evaluations.csv")
datasets = set(df['dataset'])
n_models = 2

for dataset in datasets:
    for test_fold in set(df[df['dataset'] == dataset]['test_fold']):
        
        sub_df = (
            df[(df['dataset'] == dataset) & (df['test_fold'] == test_fold)]
            .sort_values(by='mean_best_val_loss_cv')
        )
        
        # Take top models
        top = sub_df.iloc[:n_models]
        
        preds_list = []
        
        for _, conf in top.iterrows():
            pred_path = (
                f"predictions/{dataset}.{test_fold}."
                f"{conf['n_conv_layers']}."
                f"{conf['n_filters']}."
                f"{conf['kernel_size']}."
                f"{conf['n_dense_layers']}."
                f"{conf['n_neurons']}.csv"
            )
            
            pred = pd.read_csv(pred_path)
            preds_list.append(pred)
        
        # Use first file as template
        avg_pred = preds_list[0][['sequenceID']].copy()
        
        # Average only the 'pred' column
        avg_pred['pred'] = sum(p['pred'] for p in preds_list) / len(preds_list)
        
        os.makedirs("final_predictions", exist_ok=True)
        avg_pred.to_csv(f"final_predictions/{dataset}.{test_fold}.csv", index=False)